In [3]:
import copy
import sys


# class Stack(list):
#     def __init__(self):
#         self = []

#     def push_s(self, item):
#         self.append(item)

#     def pop_s(self):
#         return self.pop()

#     def is_empty(self):
#         return len(self) == 0

#     def peek(self):
#         if not self.is_empty():
#             return self[-1]

#     def get_stack(self):
#         return self


def load_graph(x):
    with open(x) as f:
        data_set = []
        for ln in f:
            line = ln.split()
            if line:
                a = [int(x) for x in line]
                data_set.append(a)
    return data_set


def list_of_zeros(n):  # function used to initialize reverse_scc-1-n sorted
    result = [[0, 0]] * n
    return result


def reverse_scc_graph_sorted(x):  # function to reverse scc graph, 1-n sorted
    rev_graph = list_of_zeros(len(x))
    for index in range(len(x)):
        node = x[index]
        rev_graph[index] = node[::-1]
    rev_graph.sort()

    return rev_graph


def compress_graph(a):  # compress data with distinct nodes
    b = [[0, 0]]
    for index in range(len(a)):
        if b[-1][0] == a[index][0]:
            b[-1].append(a[index][1])
        else:
            b.append(a[index])
    b.pop(0)
    return b


def fill_blank(m):
    for index in range(m[-1][0]):
        if not m[index][0] == index + 1:
            m.insert(index, [index + 1])
    return m


def add_init_flag(g, t_graph, checker):
    flag = False
    for x in range(0, len(g)):
        g[x].append(flag)  # add an initial flag of false(as unexplored) AT THE END of each vertex
        g[x].append(0)  # position for leader_s
        g[x].append(0)  # position for f(i) := t
        if checker == 1:
            g[x] = [len(g) - x] + g[x]  # added a label for 1-n distinct nodes
        elif checker == 2:
            g[x] = [t_graph[x][-1]] + g[x]  # added a label for 1-n distinct nodes with t

    return g


def check_flag(vertex):  # check flag of one vertex, if unexplored(false flag)
    if vertex[-3] is False:  # flag position at -3
        return 1
    else:
        return 0


def prep(graph_in, t_graph, checker):  # load in compressed scc graph data for labeling output
    copy_graph = graph_in.copy()

    stack_of_vertex = Stack()  # initiate an empty stack
    if checker == 1:
        graph_rev = reverse_scc_graph_sorted(copy_graph)
    else:
        graph_rev = copy_graph

    graph_compressed = compress_graph(graph_rev)
    graph_fill = fill_blank(graph_compressed)
    flag = add_init_flag(graph_fill, t_graph, checker)

    return flag, stack_of_vertex


def find_scc_1(graph):
    ## graph as input is the original graph
    original_graph = graph.copy()
    graph_flag, stack_of_vertex_1 = prep(graph,[],1)
    
    ## first pass loop to determin t (magic order)
    t = [0]  # number of nodes processed so far

    
    for index in range(len(graph_flag)):
        if check_flag(graph_flag[index]):
            depth_first_search_first(graph_flag,index+1,stack_of_vertex_1,t)
    
    return graph_flag
    
    ## second pass to get scc

def convert(a_list):
    return tuple(i for i in a_list)


def find_scc_2(original_graph,t_graph):
    
    leader_s = 0  # used in second pass to record label of scc
    leader_recorder = []
#     scc_lib = {}
    scc_list = []
    scc_size = 0
    
    original_t_flag, stack_of_vertex_2 = prep(original_graph,t_graph,2)
    original_flag = convert(original_t_flag)

    remove_zero = []
    
    # removed zeros as t label
    for index in range(len(original_t_flag)):
        if original_t_flag[index][0] != 0:
            remove_zero.append(original_t_flag[index])
    
    remove_zero.sort()   
    original_t_flag = remove_zero[::-1] 
    
    for index in range(len(original_t_flag)):
        if check_flag(original_t_flag[index]):
            scc_size = 0
            leader_s = original_t_flag[index][0]
            leader_recorder.append(leader_s)
            index_t_flag = len(original_t_flag) - index
            scc_size = depth_first_search_second(original_t_flag,index_t_flag,stack_of_vertex_2,leader_s,original_flag,scc_size)
        
            scc_list.append(scc_size)

        
    scc_list.sort()
    scc_list = scc_list[::-1]
    return leader_recorder, scc_list


def depth_first_search_first(graph_flag, s, stack,
                             t):  # s is the index of that vertex of interest (s-1 for index in graph)

    stack_of_vertex = stack
    

    graph_flag[s - 1][-3] = True  # mark s as explored

    # initialize Q with s
    stack_of_vertex.append(graph_flag[s - 1][0])
    length = len(graph_flag)

    if not stack_of_vertex.is_empty():
        m = stack_of_vertex[-1]
        node_m = graph_flag[length - m]

        if len(node_m) > 5:
            for index in range(2, len(node_m) - 3):
                n = node_m[index]
                if check_flag(graph_flag[n - 1]):
                    depth_first_search_first(graph_flag, n, stack_of_vertex, t)

            t[0] = t[0] + 1
            graph_flag[length - m][-1] = t[0]
            stack_of_vertex.pop()

    return


def depth_first_search_second(original_t_flag, s, stack, leader_s, original_flag,
                              scc_size):  # s is the index of that vertex of interest (s-1 for index in graph)

    stack_of_vertex = stack

    length_t = len(original_t_flag)
    index_t_flag = length_t - s


    original_t_flag[index_t_flag][-3] = True  # mark s as explored
    original_t_flag[index_t_flag][-2] = leader_s   
    scc_size += 1

    # initialize Q with s
    stack_of_vertex.append(original_t_flag[index_t_flag][0])

    if not stack_of_vertex.is_empty():
        m = stack_of_vertex[-1]
        node_m = original_t_flag[length_t - m]

        for index in range(2, len(node_m) - 3):
            n = node_m[index]
            t_flag = original_flag[n-1][0]

            if check_flag(original_t_flag[length_t-t_flag]):                
                scc_size = depth_first_search_second(original_t_flag, t_flag, stack_of_vertex, leader_s, original_flag,
                                                     scc_size)

        stack_of_vertex.pop()

    return scc_size


# if __name__ == "__main__":
#     sys.setrecursionlimit(2*10 ** 9)


#     graph = load_graph('68.txt')
#     graph_copy = graph.copy()

#     t_graph = find_scc_1(graph_copy)
#     print(t_graph[:5])
#     # recorder, scc_list = find_scc_2(graph_copy,t_graph)


#     # print(scc_list[:5])





In [5]:
%%time
if __name__ == "__main__":
    sys.setrecursionlimit(10 ** 9)


    graph = load_graph('67.txt')
    graph_copy = graph.copy()

    t_graph = find_scc_1(graph_copy)
#     print(t_graph[:5])
    recorder, scc_list = find_scc_2(graph_copy,t_graph)


    print(scc_list[:5])

[153343, 76084, 35657, 19286, 15900]
CPU times: user 15.1 s, sys: 23.5 ms, total: 15.1 s
Wall time: 15.1 s
